In [1]:
import os
os.environ["GOOGLE_API_KEY"] = "YOUR_API_KEY"

In [23]:
import google.generativeai as genai

# List available models
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-2.5-pro-exp-03-25
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash-preview-04-17
models/gemini-2.5-flash-preview-05-20
models/gemini-2.5-flash-preview-04-17-thinking
models/gemini-2.5-pro-preview-05-06
models/gemini-2.5-pro-preview-06-05
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-preview-image-generation
models/gemini-2.0-flash-lit

In [2]:
!pip install -q youtube-transcript-api langchain-community langchain-openai \
               faiss-cpu tiktoken python-dotenv langchain-google-genai google-ai-generativelanguage==0.6.15

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.5/64.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00


In [3]:
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate

## *Document Ingestion*

In [4]:
!pip install pytube openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 14.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 111.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.8 MB/s

In [5]:
!pip install youtube-dl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 29.5 MB/s eta 0:00:00


In [6]:
!pip install --upgrade pytube

In [26]:
# from pytube import YouTube

# video_url = "https://www.youtube.com/watch?v=wgfSDrqYMJ4"
# !yt-dlp -x --audio-format mp3 {video_url} -o "audio.%(ext)s"

# The downloaded file should be named "audio.mp3" in the same directory as your notebook.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.3/174.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 39.1 MB/s eta 0:00:00
[youtube] Extracting URL: https://www.youtube.com/watch?v=wgfSDrqYMJ4
[youtube] wgfSDrqYMJ4: Downloading webpage
[youtube] wgfSDrqYMJ4: Downloading tv client config
[youtube] wgfSDrqYMJ4: Downloading player 612f74a3-main
[youtube] wgfSDrqYMJ4: Downloading tv player API JSON
[youtube] wgfSDrqYMJ4: Downloading ios player API JSON
[youtube] wgfSDrqYMJ4: Downloading m3u8 information
[info] wgfSDrqYMJ4: Downloading 1 format(s): 251
[download] Destination: audio.webm
[download] 100% of    6.61MiB in 00:00:00 at 24.64MiB/s
[ExtractAudio] Destination: audio.mp3
Deleting original file audio.webm (pass -k to keep)


In [7]:
import whisper

model = whisper.load_model("base")  # Or "small", "medium", "large"
result = model.transcribe("audio.mp3")
print(result["text"])

100%|███████████████████████████████████████| 139M/139M [00:03<00:00, 47.2MiB/s]
/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


 A word, on word, embeddings. Maybe a few words. Word embeddings represent words as numbers, specifically as numeric vectors. In a way that captures their semantic relationships and contextual information. So that means words with similar meanings are positioned close to each other and the distance and direction between vectors encode the degree of similarity between words. But why do we need to transform words into numbers? The reason vectors are used to represent words is that most machine learning algorithms are just incapable of processing plain text in its raw form. They require numbers as input to perform any task. And that's where word embeddings come in. So let's take a look at how word embeddings are used and the model is used to create them. And let's start with a look at some applications. Now word embeddings have become a fundamental tool in the world of NLP. That's natural language processing. Natural language processing helps machines understand human language. Word embed

## *Text Splitting*

In [8]:
transcript = result['text']
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.create_documents([transcript])

In [9]:
len(chunks)

9

## *Embedding Generation and Storing in Vector Store*

In [10]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vector_store = FAISS.from_documents(chunks, embeddings)

In [11]:
vector_store.index_to_docstore_id

{0: 'da1a3391-debd-4c23-b3f4-7a82ec6962f9',
 1: '6be3cf4b-0550-4940-9071-8c001cc3d83b',
 2: 'eb44225e-1eea-451d-8d49-30c83e8303ff',
 3: '2a3fed4b-65bd-4c69-9850-081a0776817e',
 4: 'ffabf034-bfcb-4aa1-99c0-3813caf738a2',
 5: 'fa25385f-9cc3-42aa-86a3-5e8f4c0be137',
 6: '1e68ed8f-203d-4513-aa66-197fa704986e',
 7: '5e41d36e-376e-47a4-8875-e1d4356e2a74',
 8: '9e26f054-4a03-4449-b83f-7fc6d8db502c'}

## *Retrieval*

In [12]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 4})

In [13]:
retriever

VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x796ebc6acad0>, search_kwargs={'k': 4})

In [14]:
retriever.invoke('Applications of Word embeddings?')

[Document(id='6be3cf4b-0550-4940-9071-8c001cc3d83b', metadata={}, page_content="applications. Now word embeddings have become a fundamental tool in the world of NLP. That's natural language processing. Natural language processing helps machines understand human language. Word embeddings are used in various NLP tasks. So for example, you'll find them used in text classification very frequently. Now in text classification, word embeddings are often used in tasks such as spam detection and topic categorization. Another common task is NER, a synacronym for named entity recognition. And there is used to identify and classify entities in text. And an entity is like a name of a person or a place or an organization. Now word embeddings can also help with tasks related to word similarity and word analogy tasks. So for example, the king is dequeen as man is to woman. And then another example is in Q&A. So question and answering systems, they can benefit from word embeddings for measuring semanti

## *Augmentation*

In [24]:
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

In [16]:
prompt = PromptTemplate(
    template="""
      You are a helpful assistant.
      Answer ONLY from the provided transcript context.
      If the context is insufficient, just say you don't know.

      {context}
      Question: {question}
    """,
    input_variables = ['context', 'question']
)

In [17]:
question          = "Explain contextual based embeddings in regard of transcript"
retrieved_docs    = retriever.invoke(question)

In [18]:
retrieved_docs

[Document(id='9e26f054-4a03-4449-b83f-7fc6d8db502c', metadata={}, page_content="based embeddings are whether representation of a word changes based on its surrounding context. So for example, in a transformer model, the word bank would have different representations in the sentence, I'm going to the bank to deposit money, and I'm sitting on the bank of a river. This context sensitivity allows these models to capture more nuanced meanings and relationships between words, which has led to all sorts of improvements in the various fields of NLP tasks. So that's word embeddings from simple numeric vectors to complex representations. Word embeddings have revolutionized how machines understand and process human language, proving that transforming words into numbers is indeed a powerful tool for making sense of our linguistic world. We tab the values of анvent sorting every word that model in understanding our perception ofSPEAK point of view, lesson in meaning of calling world pens of surveyi

In [19]:
context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
context_text

"based embeddings are whether representation of a word changes based on its surrounding context. So for example, in a transformer model, the word bank would have different representations in the sentence, I'm going to the bank to deposit money, and I'm sitting on the bank of a river. This context sensitivity allows these models to capture more nuanced meanings and relationships between words, which has led to all sorts of improvements in the various fields of NLP tasks. So that's word embeddings from simple numeric vectors to complex representations. Word embeddings have revolutionized how machines understand and process human language, proving that transforming words into numbers is indeed a powerful tool for making sense of our linguistic world. We tab the values of анvent sorting every word that model in understanding our perception ofSPEAK point of view, lesson in meaning of calling world pens of surveying,\n\nhighlights words that are frequent within a specific document, but are r

In [20]:
final_prompt = prompt.invoke({"context": context_text, "question": question})

In [21]:
final_prompt

StringPromptValue(text="\n      You are a helpful assistant.\n      Answer ONLY from the provided transcript context.\n      If the context is insufficient, just say you don't know.\n\n      based embeddings are whether representation of a word changes based on its surrounding context. So for example, in a transformer model, the word bank would have different representations in the sentence, I'm going to the bank to deposit money, and I'm sitting on the bank of a river. This context sensitivity allows these models to capture more nuanced meanings and relationships between words, which has led to all sorts of improvements in the various fields of NLP tasks. So that's word embeddings from simple numeric vectors to complex representations. Word embeddings have revolutionized how machines understand and process human language, proving that transforming words into numbers is indeed a powerful tool for making sense of our linguistic world. We tab the values of анvent sorting every word that 

## *Generation*

In [25]:
answer = llm.invoke(final_prompt)
print(answer.content)

Contextual based embeddings are where the representation of a word changes based on its surrounding context. For example, in a transformer model, the word "bank" would have different representations in the sentence "I'm going to the bank to deposit money" and "I'm sitting on the bank of a river". This context sensitivity allows these models to capture more nuanced meanings and relationships between words.


# Building a Chain

In [26]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [27]:
def format_docs(retrieved_docs):
  context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
  return context_text

In [28]:
parallel_chain = RunnableParallel({
    'context': retriever | RunnableLambda(format_docs),
    'question': RunnablePassthrough()
})

In [29]:
parallel_chain.invoke('context based embedding')

{'context': "global vectors for word representation. That was created at Stanford University in 2014. And that uses co-occurrent statistics to create word vectors. Now these models, they differ in their approach. Word to veg that focuses on learning from the immediate context around each word, while glove takes a broad view by analyzing how often words appear together across the entire corpus, then uses this information to create word vectors. Now while these two word embedding models continue to be valuable tools in NLP, the field has seen some significant advances with the emergence of new tech, particularly transformers. While traditional word embeddings assign a fixed vector to each word, transformer models use a different type of embedding, and it's called a contextual based embedding. Now contextual based embeddings are whether representation of a word changes based on its surrounding context. So for example, in a transformer model, the word bank would have different representati

In [30]:
parser = StrOutputParser()

In [31]:
main_chain = parallel_chain | prompt | llm | parser

In [32]:
main_chain.invoke('Can you summarize the video')

'The video discusses word embeddings, which transform words into numbers to help machines understand and process human language. It covers different types of embeddings, including T-F-I-D-F and prediction-based embeddings, explaining how they capture semantic relationships and contextual information. The video also touches on context-sensitive embeddings, where the representation of a word changes based on its surrounding context, as seen in transformer models. It uses examples like "apple" and "orange" or "happy" and "sad" to illustrate semantic relationships and explains how models are trained to minimize prediction errors, using a small corpus of six words as an example. The video further suggests that question and answering systems can benefit from word embeddings for measuring semantic similarities between words or documents for tasks like clustering related articles or finding similar documents or recommending similar items.'